In [1]:
from datetime import date
import pandas as pd
import numpy as np
from selenium.webdriver.common.keys import Keys
from selenium import webdriver # driving the actions, links up with the browser
from bs4 import BeautifulSoup
import time
import re
from pick import pick
import progressbar
import os

In [2]:
search_webpage = 'https://waterlooworks.uwaterloo.ca/myAccount/co-op/coop-postings.htm'

PATH =  r'C:\Users\esthe\Documents\Coding\chromedriver_win32\chromedriver.exe'
driver = webdriver.Chrome(PATH)
driver.get(search_webpage)

In [56]:
yes = ["Yes", "Y", "YES", "y", "yes"]
starting_page = "https://waterlooworks.uwaterloo.ca/myAccount/co-op/coop-postings.htm"
dashboard_url = "http://waterlooworks.uwaterloo.ca/myAccount/co-op/coop-postings.htm"
login_url = 'https://cas.uwaterloo.ca/cas/login?service=https://waterlooworks.uwaterloo.ca/waterloo.htm'
output_name = 'WW_postings.csv'
choice = 'Shortlist'
potential_html1 = None
potential_html2 = None
potential_html3 = None

In [57]:
{"Work Term Duration:": None, # might need to do some regex filtering
                                         "Job Summary:" : None,
                                         "Job Responsibilities:": None,
                                         "Required Skills:" : None,
                                         "Targeted Degrees and Disciplines:": None,
                                         "Application Documents Required:": None} 

{'Work Term Duration:': None,
 'Job Summary:': None,
 'Job Responsibilities:': None,
 'Required Skills:': None,
 'Targeted Degrees and Disciplines:': None,
 'Application Documents Required:': None}

In [58]:
df = pd.DataFrame(columns=['ID', 'Job Title', 'Organization', 'Division', 'Openings', 'Internal Status', 'City', 'Level',
                           'Applications', 'App Deadline', 'Work Term Duration', "Job Summary", "Job Responsibilities", "Required Skills", 
                           "Targeted Degrees and Disciplines", "Application Documents Required"])
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required


In [122]:
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required


In [59]:
df = pd.DataFrame(columns=['ID', 'Job Title', 'Organization', 'Division', 'Openings', 'Internal Status', 'City', 'Level',
                           'Applications', 'App Deadline', 'Work Term Duration', "Job Summary", "Job Responsibilities", "Required Skills", 
                           "Targeted Degrees and Disciplines", "Application Documents Required"])

def main():
    starting_page = "https://waterlooworks.uwaterloo.ca/myAccount/co-op/coop-postings.htm"

    output_name = 'WW_postings.csv'
    
    browser = webdriver.Chrome(PATH)
    browser.get(starting_page) # in original script, this was starting page

    input('Are you ready?')

    
    data = get_job_lists(choice, browser, output_name)

    print("Done!")


def get_job_lists(choice, browser, output_name):
    
    print("Getting job lists ...")

    # Navigate to the job listings page then wait for the javascript to load
    browser.find_element_by_link_text(choice).click()

    input("Are you ready again?")
    
    # Get the HTML of the page and check if this is the last page (next_page_buttons)
    page_html = browser.execute_script("return document.body.innerHTML")
    pattern = r'<a href=".+?" onclick="loadPostingTable(.+?)">\s*»\s*<\/a>'
    next_page_buttons = []
    next_page_buttons = re.findall(pattern, page_html)



    # bar = progressbar.ProgressBar(max_value=self.job_lists_page_count)
    # bar.update(self.gather_current_progress)


    # dots for loading screen
    dots = ""
    counter_page = 0
    # Scrape the tables and save it to output_name as a CSV
    job_posting_importants_order = ['Work Term Duration:', "Job Summary:", "Job Responsibilities:", "Required Skills:", 
                                    "Targeted Degrees and Disciplines:", "Application Documents Required:"]
    with open(output_name, 'w') as f:
        row_count = 0
        job_count = 0
        while (counter_page < 4):
            counter_page += 1
            # loading screen stuff
            os.system('clear')
            print("Working" + dots)
            # update dots
            if (dots == "..."):
                dots = ""
            else:
                dots += "."

            # wait for JavaScript to load then download the page info
            time.sleep(2)
            soup = BeautifulSoup(page_html, "html.parser")
            
            for tr in soup.find_all('tr')[1:]: # iterating throw all rows
            
                
                job_count += 1
                print(job_count)
                try:
                    # all the data that was not in the original script
                    job_posting_importants = {"Work Term Duration:": None, # might need to do some regex filtering
                                             "Job Summary:" : None,
                                             "Job Responsibilities:": None,
                                             "Required Skills:" : None,
                                             "Targeted Degrees and Disciplines:": None,
                                             "Application Documents Required:": None} 


                    tds = tr.find_all('td') # iterating all cells in each row.
                    count = 0
                    job_info_list = []
                    for x in tds[0:-1]: # starting with the 3 buttons


                        if (count == 0): # at 3 buttons
                            link_container = x.find_all('a')[2]

                            onclick_link = link_container['onclick']

                            # clicking the onclick link:
                            browser.execute_script(onclick_link)
                            browser.switch_to.window(browser.window_handles[1])

                            # give the browser 1 seconds for the website to catch up with the code
                            browser.implicitly_wait(1.5)

                            time.sleep(2)
                            # do stuff, locate the texts of 'job posting importants'

                            #saving the contents of the html of the new page:
                            indiv_html = browser.execute_script("return document.body.innerHTML")
                            indiv_posting = BeautifulSoup(indiv_html, "html.parser")

                            # finding the third 'table' in the page, as the first table is no use to us

                            job_posting_information_table = indiv_posting.find_all('table', {'class': 'table-bordered'})[1]


                            
                            #print(job_posting_information_table)


                            # iterating through all rows in the table
                            for row in job_posting_information_table.find_all('tr'): 
                                cell_list = row.find_all('td')
                                try:
                                    key = cell_list[0].get_text().strip()
                                except:
                                    continue


                                if (key in job_posting_importants):
                                    try:
                                        job_posting_importants[key] = cell_list[1].get_text().strip()
                                    except:
                                        job_posting_importants[key] = ""


                            application_information_table = indiv_posting.find_all('table', {'class': 'table-bordered'})[2]
                            job_posting_importants["Application Documents Required:"] = application_information_table.find_all('td')[3].get_text().strip()

                            browser.close() # close current tab
                            browser.switch_to.window(browser.window_handles[0])

                            #print("lala")
                        if (count >= 2):  
                            #print(x.text)
                            job_info_list.append(str(x.text))

                        count += 1


                        
                    for col in job_posting_importants_order:
                        job_info_list.append(job_posting_importants[col])

                    df.loc[row_count] = job_info_list
                    row_count+=1
                except:
                    continue
                
            # navigate to the next page and get the HTML, next page buttons
            next_page_buttons = re.findall(pattern, page_html)
            browser.find_element_by_link_text("»").click()
            time.sleep(2)
            page_html = browser.execute_script("return document.body.innerHTML")


if __name__ == "__main__":
    main()

Are you ready?y
Getting job lists ...
Are you ready again?y
Working
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":"»"}
  (Session info: chrome=93.0.4577.82)


In [61]:
df[df['Work Term Duration'] != '4 month work term']

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required
17,222880,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,Daisy Intelligence Corporation,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",10,"\n\t\t\t\t\t\t\t\tSep 21, 2021\n\t\t\t\t\t\t\t...",8 month consecutive work term required,Daisy is AI done right. We deliver AI-Powered ...,The impact you will have: \n\n\nYou will be de...,What you bring:\n \n\nSolid programming skills...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résumé"
34,220360,\n\n\n\t\t\t\t\t\t\t\t\t\tML Engineering (Remo...,ezSec Inc,Divisional Office,2,Open for Applications,Kitchener,"Intermediate, Senior",27,"\n\t\t\t\t\t\t\t\tSep 21, 2021\n\t\t\t\t\t\t\t...",2 work term commitment preferred,ezSec is building easy-to-use advanced cyberse...,The position entails:\n \n\nDesigning AI solut...,Requirements:\n \n\nStrong theoretical foundat...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
48,220073,\n\n\n\t\t\t\t\t\t\t\t\t\tData Scientist Assoc...,NERv Technology Inc,Divisional Office,2,Open for Applications,Kitchener,"Intermediate, Senior",31,"\n\t\t\t\t\t\t\t\tSep 21, 2021\n\t\t\t\t\t\t\t...",8 month consecutive work term preferred,NERv aims to create a paradigm shift in post-o...,"Process, clean and verify incoming data from c...",You're self-driven and able to adapt in a fast...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
50,221085,\n\n\n\t\t\t\t\t\t\t\t\t\tData Science Analyst...,Novocol Pharmaceutical of Canada Inc,Septodont Novocol Pharmaceutical Division,1,Open for Applications,Cambridge,Junior,37,"\n\t\t\t\t\t\t\t\tSep 21, 2021\n\t\t\t\t\t\t\t...",8 month consecutive work term required,The Data Science Analyst (Co-op) position work...,PRIMARY DUTIES AND RESPONSIBILITIES:\n\n \n \n...,BASIC REQUIREMENTS:\n\nEducation\n\n Working t...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
60,222225,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,PCL Constructors Inc,PCL Industrial Management Inc,1,Open for Applications,Edmonton,"Junior, Intermediate, Senior",8,"\n\t\t\t\t\t\t\t\tSep 21, 2021\n\t\t\t\t\t\t\t...",8 month consecutive work term preferred,PCL Industrial Management is a major construct...,Roles & Responsibilities:\n\nTrain machine lea...,Skills & Qualifications:\n\nYou should be an o...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
68,215990,\n\n\n\t\t\t\t\t\t\t\t\t\tData Scientist Inter...,Symboticware Inc,Divisional Office,1,Open for Applications,Sudbury,"Junior, Intermediate",29,"\n\t\t\t\t\t\t\t\tSep 21, 2021\n\t\t\t\t\t\t\t...",8 month consecutive work term preferred,Symboticware is a fast-growing Industrial IoT ...,"As a Data Scientist Intern, you will be a cruc...","You will leverage data mining techniques, Mach...",Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
74,219275,\n\n\n\t\t\t\t\t\t\t\t\t\tDeep Learning Assist...,Vivid Machines Inc,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",27,"\n\t\t\t\t\t\t\t\tSep 21, 2021\n\t\t\t\t\t\t\t...",8 month consecutive work term preferred,"Location: Headquarters are in Toronto, but we'...",What You Will Be Doing\n\n \n \n\nData preproc...,We're looking for people with: \n\n \n\n2+ yea...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
75,219872,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,Wealthsimple,Toronto,2,Open for Applications,Toronto,"Intermediate, Senior",103,"\n\t\t\t\t\t\t\t\tSep 21, 2021\n\t\t\t\t\t\t\t...",8 month consecutive work term p

In [161]:
df.to_csv('2021-page36-37.csv')

In [68]:
#df = pd.read_csv('half_data.csv')
df2 = pd.read_csv('page6.csv')


In [73]:
df2


,Unnamed: 0,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required
0,0,185577,0 \n\n\n\n\t\t\t\t\t\t\t\t\t\tStudent Plan...,City of St Catharines,Municipal Office,1,Open for Applications,St. Catharines,"Intermediate, Senior",12,"0 \n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t...",4 month work term,0 Studen...,0 To research assigned projects for data r...,0 Students should be experienced in virtua...,0 0 Targeted Clusters\n\t\t\t\n\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
1,1,186774,0 \n\n\n\n\t\t\t\t\t\t\t\t\t\tStudent Plan...,City of Waterloo,Development Services,1,Open for Applications,Waterloo,"Intermediate, Senior",63,"0 \n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t...",4 month work term,0 Studen...,0 To research assigned projects for data r...,0 Students should be experienced in virtua...,0 0 Targeted Clusters\n\t\t\t\n\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
2,2,185901,0 \n\n\n\n\t\t\t\t\t\t\t\t\t\tStudent Plan...,City of Waterloo,Divisional Office,1,Open for Applications,Waterloo,Junior,9,"0 \n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t...",4 month work term,0 Studen...,0 To research assigned projects for data r...,0 Students should be experienced in virtua...,0 0 Targeted Clusters\n\t\t\t\n\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
3,3,186759,0 \n\n\n\n\t\t\t\t\t\t\t\t\t\tStudent Plan...,City of Waterloo,Divisional Office,1,Open for Applications,Waterloo,Junior,17,"0 \n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t...",4 month work term,0 Studen...,0 To research assigned projects for data r...,0 Students should be experienced in virtua...,0 0 Targeted Clusters\n\t\t\t\n\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
4,4,185951,0 \n\n\n\n\t\t\t\t\t\t\t\t\t\tStudent Plan...,City of Windsor,Divisional Office,1,Open for Applications,Windsor,"Junior, Intermediate, Senior",18,"0 \n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t...",4 month work term,0 Studen...,0 To research assigned projects for data r...,0 Students should be experienced in virtua...,0 0 Targeted Clusters\n\t\t\t\n\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,184464,0 \n\n\n\n\t\t\t\t\t\t\t\t\t\tStudent Plan...,Deloitte,Tax and Audit,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",7,"0 \n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t...",4 month work term,0 Studen...,0 To research assigned projects for data r...,0 Students should be experienced in virtua...,0 0 Targeted Clusters\n\t\t\t\n\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
96,96,184476,0 \n\n\n\n\t\t\t\t\t\t\t\t\t\tStudent Plan...,Deloitte,Tax and Audit,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",20,"0 \n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t...",4 month work term,0 Studen...,0 To research assigned projects for data r...,0 Students should be experienced in virtua...,0 0 Targeted Clusters\n\t\t\t\n\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
97,97,184485,0 \n\n\n\n\t\t\t\t\t\t\t\t\t\tStudent Plan...,Deloitte,Tax and Audit,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",23,"0 \n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t...",4 month work term,0 Studen...,0 To research assigned projects for data r...,0 Students should be experienced in virtua...,0 0 Targeted Clusters\n\t\t\t\n\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
98,98,184489,0 \n\n\n\n\t\t\t\t\t\t\t\t\t\tStudent Plan...,Deloitte,Tax and Audit,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",11,"0 \n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t...",4 month work term,0 Studen...,0 To research assigned projects for data r...,0 Students should be experienced in virtua...,0 0 Targeted Clusters\n\t\t\t\n\n\t\t\...,"University of Waterloo Co-op W

In [ ]:
a = {"bob": 2, 'sam':6, 'asd':123}
a.values()

In [ ]:
' '.join("j, i, s".split())

In [ ]:
e2jin@uwaterloo.ca
Eevee1218.

In [5]:
'''
Things to Change
0. Get rid of whitespaces for job title - DONE
1. Get rid of capital NEW at the start of job titles - DONE
2. get rid of divison - DONE 
3. get rid of internal status - DONE
4. turn level into 3 binary columns - Dropped it, don't think its that important
5. filter all until only app deadline <= jan 19 - DONE
6. whitespace for targeted clusters - DONE
7. App docs required turns into need cover letter? - DONE
8. change Work Term Duration to [1,4]
9. whitepsace for app deadlines -DONE

'''
df1 = pd.read_csv('page46.csv')
df2 = pd.read_csv('page17_19.csv')
df3 = pd.read_csv('page20_22.csv')
df5 = pd.read_csv('page23_25.csv')
df6 = pd.read_csv('page26_27.csv')

In [6]:
df0 = pd.read_excel("page13.xlsx")
df7 = pd.read_excel("page_11_13.xlsx")
df8 = pd.read_excel("page_7_10.xlsx")
df9 = pd.read_excel("page_14_17.xlsx")
frames = [df0, df7, df8, df9,df1, df2, df3, df5, df6]

df = pd.concat(frames)
df.drop(["Unnamed: 0"], axis=1, inplace = True)
df = df.fillna("")
df.reset_index(inplace=True)

In [7]:
df

,index,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required
0,0,185132,\n\n\n\t\t\t\t\t\t\t\t\t\tAccounting Analyst\n...,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",32,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
1,1,185136,\n\n\n\t\t\t\t\t\t\t\t\t\tSecurity\n\t\t\t\t\t...,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",87,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment.\...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
2,2,186638,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",71,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table:\n- Confidence: A ...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
3,3,186639,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",210,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table:\n- Confidence: A ...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
4,4,186640,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",47,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: \n\nConfidence: A...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,133,188060,\r\n\r\n\t\t\t\t\t\t\t\t\t\tNEW\r\n\t\t\t\t\t\...,Zoocasa,Head Office,1,Open for Applications,Toronto,"Junior, Intermediate",12,"\r\n\t\t\t\t\t\t\t\tJan 19, 2021\r\n\t\t\t\t\t...",4 month work term,"At Zoocasa, our mission is to ensure our clien...",Responsibilities:\r\n\r\n- Perform manual test...,Required Skills:\r\n- Experience with manual t...,Targeted Clusters\r\n\t\t\t\r\n\r\n\t\t\r\n\t\...,Grade Report
2511,134,185591,\r\n\r\n\r\n\r\n\t\t\t\t\t\t\t\t\t\tProtein En...,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate",7,"\r\n\t\t\t\t\t\t\t\tJan 19, 2021\r\n\t\t\t\t\t...",8 month consecutive work term required,About the Position\r\nWe are currently seeking...,Your Job \r\n\r\nSupports recombinant protein ...,"Your Academics\r\nWe want you to succeed, so y...",Targeted Clusters\r\n\t\t\t\r\n\r\n\t\t\r\n\t\...,"University of Waterloo Co-op Work History,Cove..."
2512,135,185600,\r\n\r\n\r\n\r\n\t\t\t\t\t\t\t\t\t\tComputatio...,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",3,"\r\n\t\t\t\t\t\t\t\tJan 19, 2021\r\n\t\t\t\t\t...",8 month consecutive work term required,About the Position\r\n \r\nWe are currently se...,Your Job \r\n \r\n\r\nAssists with the applica...,"Your Academics\r\n \r\nW

In [5]:
import time
from datetime import datetime

In [62]:
def surrounding_space_remover(str):
    return ' '.join(str.split())
    
df_space_remover = np.vectorize(surrounding_space_remover)

for column in ['Job Title', 'App Deadline', 'Targeted Degrees and Disciplines', "Job Summary", "Job Responsibilities", "Required Skills", "Targeted Degrees and Disciplines"]:
    df[column] = df_space_remover(df[column])

In [63]:
tic = time.time()

for column in ['Job Title', 'App Deadline', 'Targeted Degrees and Disciplines', "Job Summary", "Job Responsibilities", "Required Skills", "Targeted Degrees and Disciplines"]:
    print(column)
    df[column] = df_space_remover(df[column])
    
toc = time.time()

print(str(1000*(toc-tic)))
    

Job Title
App Deadline
Targeted Degrees and Disciplines
Job Summary
Job Responsibilities
Required Skills
Targeted Degrees and Disciplines
15.924692153930664


In [64]:
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required
0,218976,Associate ML Developer,AltaML Inc,Divisional Office,8,Open for Applications,Edmonton,Intermediate,95,"Sep 21, 2021",4 month work term,This opportunity is for current students and i...,What you'll do: Complete project related tasks...,"What you bring: Enrolled in BSc in Comp Sci, C...",Targeted Clusters - Theme - Computing: Informa...,"University of Waterloo Co-op Work History,Résu..."
1,223216,NEW Machine Learning/AI,Apple Inc,Corporate Head Office,2,Open for Applications,Remote in Canada,"Intermediate, Senior",216,"Sep 21, 2021",4 month work term,Apple is seeking highly qualified people for t...,Some responsibilities in AI/ML may include: Yo...,You may meet or have interest in any one of th...,Targeted Clusters - Theme - Computing: Informa...,"University of Waterloo Co-op Work History,Résu..."
2,220186,Data Science Co-op,Aviva Canada Inc,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",75,"Sep 21, 2021",4 month work term,"Join an exciting team of actuaries, data scien...",What you'll do Transformation of very large an...,"What you need to succeed As a data-scientist, ...",Targeted Clusters - Theme - Computing: Informa...,"University of Waterloo Co-op Work History,Résu..."
3,220275,Software Engineer/Machine Learning Engineer,Aviva Canada Inc,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",76,"Sep 21, 2021",4 month work term,"Individually we are people, but together we ar...","What you'll do Develop, test, and deploy code....",What you'll bring On-going studies in the fiel...,Targeted Clusters - Theme - Computing: Informa...,"University of Waterloo Co-op Work History,Résu..."
4,223158,NEW NLP / ML / Data Scientist at Startup,BetterUp,Divisional Office,3,Open for Applications,Remote in Canada,Senior,78,"Sep 21, 2021",4 month work term,BetterUp is a late-stage unicorn startup looki...,Job responsibilities can cover a range of data...,Engineering requirements and nice-to-haves: Al...,Targeted Clusters - Theme - Computing: Informa...,"University of Waterloo Co-op Work History,Résu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,219872,NEW Data Science Co-op (Winter 2022),Wealthsimple,Toronto,2,Open for Applications,Toronto,"Intermediate, Senior",103,"Sep 21, 2021",8 month consecutive work term preferred,Wealthsimple is on a mission to help everyone ...,We're looking for people who: Are driven to sh...,Things you should know about our tech before a...,"Targeted Clusters - Theme - Data Analytics, St...","University of Waterloo Co-op Work History,Résu..."
76,221949,NEW AI/ML Developer Intern,Wind River,Ottawa Office,1,Open for Applications,Ottawa,"Junior, Intermediate, Senior",50,"Sep 21, 2021",4 month work term,Wind River believes that some things are so im...,What you will have an opportunity to do: • Ext...,Education & Experience: You will need: • Cours...,Targeted Clusters - Theme - Computing: Informa...,"University of Waterloo Co-op Work History,Résu..."
77,221877,NEW Data Science Associate,XE.com,Divisional Office,1,Open for Applications,Newmarket,"Intermediate, Senior",10,"Sep 21, 2021",4 month work term,The Opportunity This is a unique and exciting ...,Job Responsibilities: You will help build the ...,"Required Knowledge, Skills and Abilities Fluen...","Targeted Clusters - Theme - Data Analytics, St...","University of Waterloo Co-op Work History,Cove..."
78,221274,Machine Learning Intern,XSENSOR Technology Corporation,Corporate Head Office,1,Open for Applications,Calgary,Junior,96,"Sep 21, 2021",4 month work term,We are looking for a hard-working and motivate...,This student job opportunity will primarily in...,- Familiar with Python and Tensorflow. - Detai...,Targeted Clusters - Theme - Medical Device

In [65]:

def surrounding_space_remover(str):
    return ' '.join(str.split())
    
df_space_remover = np.vectorize(surrounding_space_remover)

In [66]:
tic = time.time()
for column in ['Job Title', 'App Deadline', 'Targeted Degrees and Disciplines', "Job Summary", "Job Responsibilities", "Required Skills", "Targeted Degrees and Disciplines"]:
    for i in range(len(df[column])):
        df.at[i, column] = surrounding_space_remover(df.at[i,column])
toc = time.time()

print(str(1000*(toc-tic)))
    

10.9710693359375


In [67]:
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required
0,218976,Associate ML Developer,AltaML Inc,Divisional Office,8,Open for Applications,Edmonton,Intermediate,95,"Sep 21, 2021",4 month work term,This opportunity is for current students and i...,What you'll do: Complete project related tasks...,"What you bring: Enrolled in BSc in Comp Sci, C...",Targeted Clusters - Theme - Computing: Informa...,"University of Waterloo Co-op Work History,Résu..."
1,223216,NEW Machine Learning/AI,Apple Inc,Corporate Head Office,2,Open for Applications,Remote in Canada,"Intermediate, Senior",216,"Sep 21, 2021",4 month work term,Apple is seeking highly qualified people for t...,Some responsibilities in AI/ML may include: Yo...,You may meet or have interest in any one of th...,Targeted Clusters - Theme - Computing: Informa...,"University of Waterloo Co-op Work History,Résu..."
2,220186,Data Science Co-op,Aviva Canada Inc,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",75,"Sep 21, 2021",4 month work term,"Join an exciting team of actuaries, data scien...",What you'll do Transformation of very large an...,"What you need to succeed As a data-scientist, ...",Targeted Clusters - Theme - Computing: Informa...,"University of Waterloo Co-op Work History,Résu..."
3,220275,Software Engineer/Machine Learning Engineer,Aviva Canada Inc,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",76,"Sep 21, 2021",4 month work term,"Individually we are people, but together we ar...","What you'll do Develop, test, and deploy code....",What you'll bring On-going studies in the fiel...,Targeted Clusters - Theme - Computing: Informa...,"University of Waterloo Co-op Work History,Résu..."
4,223158,NEW NLP / ML / Data Scientist at Startup,BetterUp,Divisional Office,3,Open for Applications,Remote in Canada,Senior,78,"Sep 21, 2021",4 month work term,BetterUp is a late-stage unicorn startup looki...,Job responsibilities can cover a range of data...,Engineering requirements and nice-to-haves: Al...,Targeted Clusters - Theme - Computing: Informa...,"University of Waterloo Co-op Work History,Résu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,219872,NEW Data Science Co-op (Winter 2022),Wealthsimple,Toronto,2,Open for Applications,Toronto,"Intermediate, Senior",103,"Sep 21, 2021",8 month consecutive work term preferred,Wealthsimple is on a mission to help everyone ...,We're looking for people who: Are driven to sh...,Things you should know about our tech before a...,"Targeted Clusters - Theme - Data Analytics, St...","University of Waterloo Co-op Work History,Résu..."
76,221949,NEW AI/ML Developer Intern,Wind River,Ottawa Office,1,Open for Applications,Ottawa,"Junior, Intermediate, Senior",50,"Sep 21, 2021",4 month work term,Wind River believes that some things are so im...,What you will have an opportunity to do: • Ext...,Education & Experience: You will need: • Cours...,Targeted Clusters - Theme - Computing: Informa...,"University of Waterloo Co-op Work History,Résu..."
77,221877,NEW Data Science Associate,XE.com,Divisional Office,1,Open for Applications,Newmarket,"Intermediate, Senior",10,"Sep 21, 2021",4 month work term,The Opportunity This is a unique and exciting ...,Job Responsibilities: You will help build the ...,"Required Knowledge, Skills and Abilities Fluen...","Targeted Clusters - Theme - Data Analytics, St...","University of Waterloo Co-op Work History,Cove..."
78,221274,Machine Learning Intern,XSENSOR Technology Corporation,Corporate Head Office,1,Open for Applications,Calgary,Junior,96,"Sep 21, 2021",4 month work term,We are looking for a hard-working and motivate...,This student job opportunity will primarily in...,- Familiar with Python and Tensorflow. - Detai...,Targeted Clusters - Theme - Medical Device

In [185]:
column

'Job Title'

In [186]:
df.at[1, "Job Title"]

array(['\n\n\n\t\t\t\t\t\t\t\t\t\tSecurity\n\t\t\t\t\t\t\t\t\t\n\n\n',
       '\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\t\n\n\n\t\t\t\t\t\t\t\t\t\tSummer Housekeeping Staff\n\t\t\t\t\t\t\t\t\t\n\n\n',
       '\n\n\n\n\t\t\t\t\t\t\t\t\t\tFinancial Analyst, Financial Planning & Business Performance\n\t\t\t\t\t\t\t\t\t\n\n\n\n',
       '\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\t\n\n\n\t\t\t\t\t\t\t\t\t\tOptical Algorithms Engineering\n\t\t\t\t\t\t\t\t\t\n\n\n',
       '\n\n\n\t\t\t\t\t\t\t\t\t\tSecurity\n\t\t\t\t\t\t\t\t\t\n\n\n',
       '\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\t\n\n\n\t\t\t\t\t\t\t\t\t\tSummer Housekeeping Staff\n\t\t\t\t\t\t\t\t\t\n\n\n',
       '\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\t\n\n\n\t\t\t\t\t\t\t\t\t\tOptical Algorithms Engineering\n\t\t\t\t\t\t\t\t\t\n\n\n',
       '\n\n\n\n\t\t\t\t\t\t\t\t\t\tFinancial Analyst, Financial Planning & Business Performance\n\t\t\t\t\t\t\t\t\t\n\n\n\n',
       '\n\n\n\t\t\t\t\t\t\t\t\t\tSecurity\n\t\t\t\t\t

In [37]:
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required
0,185132,Accounting Analyst,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",32,"Jan 19, 2021",4 month work term,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters - Theme - Accounting and Fin...,"University of Waterloo Co-op Work History,Résu..."
1,185136,Security,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",87,"Jan 19, 2021",4 month work term,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment. ...,Targeted Clusters - Theme - Computing: Softwar...,"University of Waterloo Co-op Work History,Résu..."
2,186638,NEW QA Intern,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",71,"Jan 19, 2021",4 month work term,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Quality...,"University of Waterloo Co-op Work History,Résu..."
3,186639,NEW Junior Developer,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",210,"Jan 19, 2021",4 month work term,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Informa...,"University of Waterloo Co-op Work History,Résu..."
4,186640,NEW 1Password for Linux,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",47,"Jan 19, 2021",4 month work term,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: Confidence: A wil...,Targeted Clusters - Theme - Computing: Softwar...,"University of Waterloo Co-op Work History,Résu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,186915,NEW Product Development Co-op,O2 Industries,Product Development,3,Open for Applications,Kitchener,"Junior, Intermediate, Senior",77,"Jan 19, 2021",4 month work term,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Support research a...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters - Theme - Human Factors and ...,"University of Waterloo Co-op Work History,Résu..."
296,186918,NEW Product Analyst Co-op,O2 Industries,Product Development,1,Open for Applications,Kitchener,"Junior, Intermediate, Senior",45,"Jan 19, 2021",4 month work term,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Support product ma...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters ARTS - Business ENG - Manage...,"University of Waterloo Co-op Work History,Résu..."
297,186921,NEW Finance Co-op,O2 Industries,Product Development,1,Open for Applications,Kitchener,"Junior, Intermediate, Senior",28,"Jan 19, 2021",4 month work term,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Support the execut...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters ARTS - Business ARTS - Econo...,"University of Waterloo Co-op Work History,Résu..."
298,186923,NEW eCommerce Marketing Co-op,O2 Industries,Product Development,1,Open for Applications,Kitchener,"Junior, Intermediate, Senior",20,"Jan 19, 2021",4 month work term,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Work cross-functio...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters - Theme - Business Administr...,"University of Waterloo Co-op 

In [68]:
def new_removal(str):
    try:
        if (str[0:3] == "NEW"): 
            return str[3:]
    except:
        return str # not enough chars in job title
    return str # if no NEW in job title

df_new_remover = np.vectorize(new_removal)

In [69]:
df['Job Title'] = df_new_remover(df["Job Title"])

In [56]:
def day_extractor(str):
    str = str.replace(",", " ")
    str = [int(s) for s in str.split() if s.isdigit()]
    return str[0]

df_day_extractor = np.vectorize(day_extractor)

df['App Deadline'] = df_day_extractor(df["App Deadline"])
#adf = df[df["App Deadline"] <= 19]


In [13]:
df_copy = df.copy()

In [20]:
df = df_copy.copy()

In [70]:
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required
0,218976,Associate ML Developer,AltaML Inc,Divisional Office,8,Open for Applications,Edmonton,Intermediate,95,"Sep 21, 2021",4 month work term,This opportunity is for current students and i...,What you'll do: Complete project related tasks...,"What you bring: Enrolled in BSc in Comp Sci, C...",Targeted Clusters - Theme - Computing: Informa...,"University of Waterloo Co-op Work History,Résu..."
1,223216,Machine Learning/AI,Apple Inc,Corporate Head Office,2,Open for Applications,Remote in Canada,"Intermediate, Senior",216,"Sep 21, 2021",4 month work term,Apple is seeking highly qualified people for t...,Some responsibilities in AI/ML may include: Yo...,You may meet or have interest in any one of th...,Targeted Clusters - Theme - Computing: Informa...,"University of Waterloo Co-op Work History,Résu..."
2,220186,Data Science Co-op,Aviva Canada Inc,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",75,"Sep 21, 2021",4 month work term,"Join an exciting team of actuaries, data scien...",What you'll do Transformation of very large an...,"What you need to succeed As a data-scientist, ...",Targeted Clusters - Theme - Computing: Informa...,"University of Waterloo Co-op Work History,Résu..."
3,220275,Software Engineer/Machine Learning Engineer,Aviva Canada Inc,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",76,"Sep 21, 2021",4 month work term,"Individually we are people, but together we ar...","What you'll do Develop, test, and deploy code....",What you'll bring On-going studies in the fiel...,Targeted Clusters - Theme - Computing: Informa...,"University of Waterloo Co-op Work History,Résu..."
4,223158,NLP / ML / Data Scientist at Startup,BetterUp,Divisional Office,3,Open for Applications,Remote in Canada,Senior,78,"Sep 21, 2021",4 month work term,BetterUp is a late-stage unicorn startup looki...,Job responsibilities can cover a range of data...,Engineering requirements and nice-to-haves: Al...,Targeted Clusters - Theme - Computing: Informa...,"University of Waterloo Co-op Work History,Résu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,219872,Data Science Co-op (Winter 2022),Wealthsimple,Toronto,2,Open for Applications,Toronto,"Intermediate, Senior",103,"Sep 21, 2021",8 month consecutive work term preferred,Wealthsimple is on a mission to help everyone ...,We're looking for people who: Are driven to sh...,Things you should know about our tech before a...,"Targeted Clusters - Theme - Data Analytics, St...","University of Waterloo Co-op Work History,Résu..."
76,221949,AI/ML Developer Intern,Wind River,Ottawa Office,1,Open for Applications,Ottawa,"Junior, Intermediate, Senior",50,"Sep 21, 2021",4 month work term,Wind River believes that some things are so im...,What you will have an opportunity to do: • Ext...,Education & Experience: You will need: • Cours...,Targeted Clusters - Theme - Computing: Informa...,"University of Waterloo Co-op Work History,Résu..."
77,221877,Data Science Associate,XE.com,Divisional Office,1,Open for Applications,Newmarket,"Intermediate, Senior",10,"Sep 21, 2021",4 month work term,The Opportunity This is a unique and exciting ...,Job Responsibilities: You will help build the ...,"Required Knowledge, Skills and Abilities Fluen...","Targeted Clusters - Theme - Data Analytics, St...","University of Waterloo Co-op Work History,Cove..."
78,221274,Machine Learning Intern,XSENSOR Technology Corporation,Corporate Head Office,1,Open for Applications,Calgary,Junior,96,"Sep 21, 2021",4 month work term,We are looking for a hard-working and motivate...,This student job opportunity will primarily in...,- Familiar with Python and Tensorflow. - Detai...,Targeted Clusters - Theme - Medical Devices an...,"University 

In [29]:
df

,index,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required
0,0,185132,Accounting Analyst,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",32,"Jan 19, 2021",4 month work term,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters - Theme - Accounting and Fin...,"University of Waterloo Co-op Work History,Résu..."
1,1,185136,Security,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",87,"Jan 19, 2021",4 month work term,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment. ...,Targeted Clusters - Theme - Computing: Softwar...,"University of Waterloo Co-op Work History,Résu..."
2,2,186638,NEW QA Intern,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",71,"Jan 19, 2021",4 month work term,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Quality...,"University of Waterloo Co-op Work History,Résu..."
3,3,186639,NEW Junior Developer,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",210,"Jan 19, 2021",4 month work term,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Informa...,"University of Waterloo Co-op Work History,Résu..."
4,4,186640,NEW 1Password for Linux,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",47,"Jan 19, 2021",4 month work term,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: Confidence: A wil...,Targeted Clusters - Theme - Computing: Softwar...,"University of Waterloo Co-op Work History,Résu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,133,188060,NEW QA Automation Developer,Zoocasa,Head Office,1,Open for Applications,Toronto,"Junior, Intermediate",12,"Jan 19, 2021",4 month work term,"At Zoocasa, our mission is to ensure our clien...",Responsibilities: - Perform manual testing - C...,Required Skills: - Experience with manual test...,Targeted Clusters - Theme - Computing: Softwar...,Grade Report
2511,134,185591,Protein Engineering Lab,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate",7,"Jan 19, 2021",8 month consecutive work term required,About the Position We are currently seeking a ...,Your Job Supports recombinant protein producti...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Scientific Experim...,"University of Waterloo Co-op Work History,Cove..."
2512,135,185600,Computational Modelling,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",3,"Jan 19, 2021",8 month consecutive work term required,About the Position We are currently seeking a ...,Your Job Assists with the application and deve...,"Your Academics We want you to succeed, so you'...","Targeted Clusters - Theme - Data Analytics, St...","University of Waterloo Co-op Work History,Cove..."
2513,136,185603,Software Development,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",4,"Jan 19, 2021",8 month consecutive work term preferred,About the Position Zymeworks Inc. is proud to ...,Your Job Work on a key software project in the...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Computing: Softwar...,"Univers

In [71]:
def cover_letter_needed(mystr):
    if (type(mystr) is not str): 
        print("OH NO!")
        print(type(mystr))
        print(str)
    return ("Cover Letter" in mystr)

df_cover_letter_needed = np.vectorize(cover_letter_needed)

In [72]:
df["Application Documents Required"] = df_cover_letter_needed(df["Application Documents Required"])

In [76]:
df.to_csv('my_shortlist.csv')

In [74]:
df.rename(columns={"Application Documents Required": "Cover Letter Needed"}, inplace=True)

In [59]:
def work_term_duration(str):
    if (str == '4 month work term'): return 1
    if (str == '8 month consecutive work term required'): return 2
    if (str == '8 month consecutive work term preferred'): return 3
    if (str == '2 work term commitment preferred'): return 4
    if (str == '2 work term commitment required'): return 5
    else: return 6 # in case theres smth weird
    
df_work_term_duration = np.vectorize(work_term_duration)

In [60]:
df["Work Term Duration"] = df_work_term_duration(df["Work Term Duration"])

<ipython-input-60-f926741de716>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Work Term Duration"] = df_work_term_duration(df["Work Term Duration"])


In [61]:
df


,index,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed
0,0,185132,Accounting Analyst,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",32,19,1,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters - Theme - Accounting and Fin...,False
1,1,185136,Security,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",87,19,1,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment. ...,Targeted Clusters - Theme - Computing: Softwar...,False
2,2,186638,QA Intern,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",71,19,1,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Quality...,False
3,3,186639,Junior Developer,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",210,19,1,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Informa...,False
4,4,186640,1Password for Linux,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",47,19,1,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: Confidence: A wil...,Targeted Clusters - Theme - Computing: Softwar...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,133,188060,QA Automation Developer,Zoocasa,Head Office,1,Open for Applications,Toronto,"Junior, Intermediate",12,19,1,"At Zoocasa, our mission is to ensure our clien...",Responsibilities: - Perform manual testing - C...,Required Skills: - Experience with manual test...,Targeted Clusters - Theme - Computing: Softwar...,False
2511,134,185591,Protein Engineering Lab,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate",7,19,2,About the Position We are currently seeking a ...,Your Job Supports recombinant protein producti...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Scientific Experim...,True
2512,135,185600,Computational Modelling,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",3,19,2,About the Position We are currently seeking a ...,Your Job Assists with the application and deve...,"Your Academics We want you to succeed, so you'...","Targeted Clusters - Theme - Data Analytics, St...",True
2513,136,185603,Software Development,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",4,19,3,About the Position Zymeworks Inc. is proud to ...,Your Job Work on a key software project in the...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Computing: Softwar...,True


In [9]:
df_710 = pd.read_excel("page_7_10.xlsx").drop(['Unnamed: 0'], axis=1)

In [10]:
df_710

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required
0,184492,\n\n\n\n\t\t\t\t\t\t\t\t\t\tAnalyst Intern - I...,Deloitte,Tax and Audit,1,Open for Applications,NaN,"Junior, Intermediate, Senior",29,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,Partner with clients to solve their most compl...,About the team \n \nHaving advised on many of ...,"Enough about us, let's talk about you. \n \nYo...",Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
1,185906,"\n\n\n\n\t\t\t\t\t\t\t\t\t\tFinancial Analyst,...",Deloitte,Tax and Audit,1,Open for Applications,Toronto,Junior,20,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",8 month consecutive work term required,"Enjoy flexible, proactive, and practical benef...",What will your typical day look like?\n\nAs pa...,"Enough about us, let's talk about you\n \nYou ...",Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
2,185909,"\n\n\n\n\t\t\t\t\t\t\t\t\t\tFinancial Analyst,...",Deloitte,Tax and Audit,1,Open for Applications,Toronto,Junior,5,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",8 month consecutive work term required,Learn from deep subject matter experts through...,What will your typical day look like?\n\nThis ...,"Enough about us, let's talk about you\n \nYou ...",Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
3,186059,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,Deloitte,Tax and Audit,10,Open for Applications,NaN,Junior,41,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,The ARA practice is a multi-disciplinary and c...,"Our Actuarial, Rewards & Analytics (ARA) pract...",Working towards a Bachelor's degree from a lea...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
4,187280,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,Deloitte,Tax and Audit,7,Open for Applications,Toronto,"Junior, Intermediate",22,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,Build a network of colleagues for life.\nBe ex...,What will your typical day look like? \n\nAs p...,"Enough about us, let's talk about you\n\nYou a...",Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,184219,\n\n\n\n\t\t\t\t\t\t\t\t\t\tData Analyst Assoc...,Geotab Inc,Oakville,1,Open for Applications,Oakville,"Junior, Intermediate, Senior",23,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",8 month consecutive work term required,Who we are:\n \nGeotab is a global leader in I...,Duties and Tasks/Essential Functions:\n\n- Col...,Experience/Skills Required: \n- Completing a U...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,Résumé
296,184220,"\n\n\n\n\t\t\t\t\t\t\t\t\t\tData Analyst, Cust...",Geotab Inc,Oakville,1,Open for Applications,Oakville,"Junior, Intermediate, Senior",17,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",8 month consecutive work term required,Who we are:\n \nGeotab is a global leader in I...,Your Responsibilities:\n\n- Perform data colle...,Your Qualifications:\n\n- Completing a degree ...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,Résumé
297,184224,\n\n\n\t\t\t\t\t\t\t\t\t\tDatabase Administrat...,Geotab Inc,Oakville,1,Open for Applications,Oakville,"Junior, Intermediate, Senior",7,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,Who we are:\n\nGeotab is a global leader in Io...,Duties and Tasks/Essential Functions:\n\n- Per...,Experience/Skills Required:\n\n- Completing a ...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,Résumé
298,184226,\n\n\n\n\t\t\t\t\t\t\t\t\t\tProduct Management...,Geo

In [48]:
esthers_csv = pd.read_csv("esthersfinal.csv").drop(["Unnamed: 0"], axis=1)

In [49]:
esthers_csv

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed
0,186082,Data Scientist Student - 4 Month Summer Term,BlackBerry,Head Office,1,Open for Applications,Waterloo,"Junior, Intermediate, Senior",75,19,1,POSITION SUMMARY This role is in BlackBerry Ad...,RESPONSIBILITIES Develop novel ML capabilities...,SKILLS AND QUALIFICATIONS Currently enrolled i...,Targeted Clusters - Theme - Computing: Softwar...,False
1,186083,Business Process Analyst Student - 4 Month Su...,BlackBerry,Head Office,1,Open for Applications,Waterloo,"Junior, Intermediate, Senior",69,19,1,POSITION SUMMARY Our Business Process Analyst ...,RESPONSIBILITIES · Creating and fostering prof...,SKILLS AND QUALIFICATIONS · Currently enrolled...,Targeted Clusters - Theme - Business Administr...,False
2,186087,Software Development Student - 8 Month Summerr...,BlackBerry,Head Office,1,Open for Applications,Mississauga,"Junior, Intermediate, Senior",3,19,2,"POSITION SUMMARY If you are an ""out of the box...",RESPONSIBILITIES Design and implementation of ...,SKILLS AND QUALIFICATIONS Good knowledge of Ob...,Targeted Clusters - Theme - Computing: Softwar...,False
3,186091,Software Development Student - 4 Month Summer ...,BlackBerry,Head Office,1,Open for Applications,Mississauga,"Junior, Intermediate, Senior",40,19,1,POSITION SUMMARY BlackBerry Advanced Technolog...,RESPONSIBILITIES Work in a cross-functional te...,SKILLS AND QUALIFICATIONS Currently enrolled i...,Targeted Clusters - Theme - Computing: Softwar...,False
4,186095,Security Response Program Manager Student - 4 ...,BlackBerry,Head Office,1,Open for Applications,Waterloo,"Junior, Intermediate",23,19,1,POSITION SUMMARY BlackBerry strives to build a...,RESPONSIBILITIES Act as an organizational foca...,SKILLS AND QUALIFICATIONS Currently enrolled i...,Targeted Clusters - Theme - Computing: Informa...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,188060,QA Automation Developer,Zoocasa,Head Office,1,Open for Applications,Toronto,"Junior, Intermediate",12,19,1,"At Zoocasa, our mission is to ensure our clien...",Responsibilities: - Perform manual testing - C...,Required Skills: - Experience with manual test...,Targeted Clusters - Theme - Computing: Softwar...,False
1596,185591,Protein Engineering Lab,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate",7,19,2,About the Position We are currently seeking a ...,Your Job Supports recombinant protein producti...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Scientific Experim...,False
1597,185600,Computational Modelling,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",3,19,2,About the Position We are currently seeking a ...,Your Job Assists with the application and deve...,"Your Academics We want you to succeed, so you'...","Targeted Clusters - Theme - Data Analytics, St...",False
1598,185603,Software Development,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",4,19,3,About the Position Zymeworks Inc. is proud to ...,Your Job Work on a key software project in the...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Computing: Softwar...,False


In [55]:
df = df.reset_index().drop(["index", "level_0"], axis=1)

In [56]:
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed
0,185132,Accounting Analyst,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",32,19,1,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters - Theme - Accounting and Fin...,False
1,185136,Security,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",87,19,1,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment. ...,Targeted Clusters - Theme - Computing: Softwar...,False
2,186638,QA Intern,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",71,19,1,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Quality...,False
3,186639,Junior Developer,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",210,19,1,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Informa...,False
4,186640,1Password for Linux,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",47,19,1,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: Confidence: A wil...,Targeted Clusters - Theme - Computing: Softwar...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,186915,Product Development Co-op,O2 Industries,Product Development,3,Open for Applications,Kitchener,"Junior, Intermediate, Senior",77,19,1,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Support research a...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters - Theme - Human Factors and ...,False
1196,186918,Product Analyst Co-op,O2 Industries,Product Development,1,Open for Applications,Kitchener,"Junior, Intermediate, Senior",45,19,1,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Support product ma...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters ARTS - Business ENG - Manage...,False
1197,186921,Finance Co-op,O2 Industries,Product Development,1,Open for Applications,Kitchener,"Junior, Intermediate, Senior",28,19,1,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Support the execut...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters ARTS - Business ARTS - Econo...,False
1198,186923,eCommerce Marketing Co-op,O2 Industries,Product Development,1,Open for Applications,Kitchener,"Junior, Intermediate, Senior",20,19,1,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Work cross-functio...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters - Theme - Business Administr...,False


In [57]:
frames = [esthers_csv, df]
df = pd.concat(frames)

In [58]:
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed
0,186082,Data Scientist Student - 4 Month Summer Term,BlackBerry,Head Office,1,Open for Applications,Waterloo,"Junior, Intermediate, Senior",75,19,1,POSITION SUMMARY This role is in BlackBerry Ad...,RESPONSIBILITIES Develop novel ML capabilities...,SKILLS AND QUALIFICATIONS Currently enrolled i...,Targeted Clusters - Theme - Computing: Softwar...,False
1,186083,Business Process Analyst Student - 4 Month Su...,BlackBerry,Head Office,1,Open for Applications,Waterloo,"Junior, Intermediate, Senior",69,19,1,POSITION SUMMARY Our Business Process Analyst ...,RESPONSIBILITIES · Creating and fostering prof...,SKILLS AND QUALIFICATIONS · Currently enrolled...,Targeted Clusters - Theme - Business Administr...,False
2,186087,Software Development Student - 8 Month Summerr...,BlackBerry,Head Office,1,Open for Applications,Mississauga,"Junior, Intermediate, Senior",3,19,2,"POSITION SUMMARY If you are an ""out of the box...",RESPONSIBILITIES Design and implementation of ...,SKILLS AND QUALIFICATIONS Good knowledge of Ob...,Targeted Clusters - Theme - Computing: Softwar...,False
3,186091,Software Development Student - 4 Month Summer ...,BlackBerry,Head Office,1,Open for Applications,Mississauga,"Junior, Intermediate, Senior",40,19,1,POSITION SUMMARY BlackBerry Advanced Technolog...,RESPONSIBILITIES Work in a cross-functional te...,SKILLS AND QUALIFICATIONS Currently enrolled i...,Targeted Clusters - Theme - Computing: Softwar...,False
4,186095,Security Response Program Manager Student - 4 ...,BlackBerry,Head Office,1,Open for Applications,Waterloo,"Junior, Intermediate",23,19,1,POSITION SUMMARY BlackBerry strives to build a...,RESPONSIBILITIES Act as an organizational foca...,SKILLS AND QUALIFICATIONS Currently enrolled i...,Targeted Clusters - Theme - Computing: Informa...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,186915,Product Development Co-op,O2 Industries,Product Development,3,Open for Applications,Kitchener,"Junior, Intermediate, Senior",77,19,1,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Support research a...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters - Theme - Human Factors and ...,False
1196,186918,Product Analyst Co-op,O2 Industries,Product Development,1,Open for Applications,Kitchener,"Junior, Intermediate, Senior",45,19,1,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Support product ma...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters ARTS - Business ENG - Manage...,False
1197,186921,Finance Co-op,O2 Industries,Product Development,1,Open for Applications,Kitchener,"Junior, Intermediate, Senior",28,19,1,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Support the execut...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters ARTS - Business ARTS - Econo...,False
1198,186923,eCommerce Marketing Co-op,O2 Industries,Product Development,1,Open for Applications,Kitchener,"Junior, Intermediate, Senior",20,19,1,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Work cross-functio...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters - Theme - Business Administr...,False


In [69]:
df = df.drop(["index"], axis=1)

In [70]:
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed
0,185132,Accounting Analyst,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",32,19,1,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters - Theme - Accounting and Fin...,False
1,185136,Security,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",87,19,1,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment. ...,Targeted Clusters - Theme - Computing: Softwar...,False
2,186638,QA Intern,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",71,19,1,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Quality...,False
3,186639,Junior Developer,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",210,19,1,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Informa...,False
4,186640,1Password for Linux,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",47,19,1,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: Confidence: A wil...,Targeted Clusters - Theme - Computing: Softwar...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,188060,QA Automation Developer,Zoocasa,Head Office,1,Open for Applications,Toronto,"Junior, Intermediate",12,19,1,"At Zoocasa, our mission is to ensure our clien...",Responsibilities: - Perform manual testing - C...,Required Skills: - Experience with manual test...,Targeted Clusters - Theme - Computing: Softwar...,False
2497,185591,Protein Engineering Lab,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate",7,19,2,About the Position We are currently seeking a ...,Your Job Supports recombinant protein producti...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Scientific Experim...,True
2498,185600,Computational Modelling,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",3,19,2,About the Position We are currently seeking a ...,Your Job Assists with the application and deve...,"Your Academics We want you to succeed, so you'...","Targeted Clusters - Theme - Data Analytics, St...",True
2499,185603,Software Development,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",4,19,3,About the Position Zymeworks Inc. is proud to ...,Your Job Work on a key software project in the...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Computing: Softwar...,True


In [62]:
df1 = df.duplicated(subset=['ID'])
df1

0       False
1       False
2       False
3       False
4       False
        ...  
2795    False
2796    False
2797    False
2798    False
2799    False
Length: 2800, dtype: bool

In [64]:
df1 = df1[df1 == True]
df1

864     True
865     True
866     True
867     True
868     True
        ... 
1595    True
1596    True
1597    True
1598    True
1599    True
Length: 318, dtype: bool

In [65]:
df.iloc[867]

ID                                                                             186844
Job Title                                           Engineering Assistant - Vancouver
Organization                                                 RDH Building Science Inc
Division                                                                  Head Office
Openings                                                                            1
Internal Status                                                 Open for Applications
City                                                                          Burnaby
Level                                                            Intermediate, Senior
Applications                                                                       10
App Deadline                                                                       19
Work Term Duration                                                                  1
Job Summary                         RDH is an employee

In [71]:
df.drop_duplicates(subset=['ID'], inplace=True)

In [70]:
df = df.reset_index().drop(["index"], axis=1)
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed
0,186082,Data Scientist Student - 4 Month Summer Term,BlackBerry,Head Office,1,Open for Applications,Waterloo,"Junior, Intermediate, Senior",75,19,1,POSITION SUMMARY This role is in BlackBerry Ad...,RESPONSIBILITIES Develop novel ML capabilities...,SKILLS AND QUALIFICATIONS Currently enrolled i...,Targeted Clusters - Theme - Computing: Softwar...,False
1,186083,Business Process Analyst Student - 4 Month Su...,BlackBerry,Head Office,1,Open for Applications,Waterloo,"Junior, Intermediate, Senior",69,19,1,POSITION SUMMARY Our Business Process Analyst ...,RESPONSIBILITIES · Creating and fostering prof...,SKILLS AND QUALIFICATIONS · Currently enrolled...,Targeted Clusters - Theme - Business Administr...,False
2,186087,Software Development Student - 8 Month Summerr...,BlackBerry,Head Office,1,Open for Applications,Mississauga,"Junior, Intermediate, Senior",3,19,2,"POSITION SUMMARY If you are an ""out of the box...",RESPONSIBILITIES Design and implementation of ...,SKILLS AND QUALIFICATIONS Good knowledge of Ob...,Targeted Clusters - Theme - Computing: Softwar...,False
3,186091,Software Development Student - 4 Month Summer ...,BlackBerry,Head Office,1,Open for Applications,Mississauga,"Junior, Intermediate, Senior",40,19,1,POSITION SUMMARY BlackBerry Advanced Technolog...,RESPONSIBILITIES Work in a cross-functional te...,SKILLS AND QUALIFICATIONS Currently enrolled i...,Targeted Clusters - Theme - Computing: Softwar...,False
4,186095,Security Response Program Manager Student - 4 ...,BlackBerry,Head Office,1,Open for Applications,Waterloo,"Junior, Intermediate",23,19,1,POSITION SUMMARY BlackBerry strives to build a...,RESPONSIBILITIES Act as an organizational foca...,SKILLS AND QUALIFICATIONS Currently enrolled i...,Targeted Clusters - Theme - Computing: Informa...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2477,186915,Product Development Co-op,O2 Industries,Product Development,3,Open for Applications,Kitchener,"Junior, Intermediate, Senior",77,19,1,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Support research a...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters - Theme - Human Factors and ...,False
2478,186918,Product Analyst Co-op,O2 Industries,Product Development,1,Open for Applications,Kitchener,"Junior, Intermediate, Senior",45,19,1,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Support product ma...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters ARTS - Business ENG - Manage...,False
2479,186921,Finance Co-op,O2 Industries,Product Development,1,Open for Applications,Kitchener,"Junior, Intermediate, Senior",28,19,1,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Support the execut...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters ARTS - Business ARTS - Econo...,False
2480,186923,eCommerce Marketing Co-op,O2 Industries,Product Development,1,Open for Applications,Kitchener,"Junior, Intermediate, Senior",20,19,1,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Work cross-functio...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters - Theme - Business Administr...,False


In [72]:
df["would_shortlist"] = False

In [72]:
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed
0,185132,Accounting Analyst,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",32,19,1,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters - Theme - Accounting and Fin...,False
1,185136,Security,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",87,19,1,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment. ...,Targeted Clusters - Theme - Computing: Softwar...,False
2,186638,QA Intern,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",71,19,1,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Quality...,False
3,186639,Junior Developer,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",210,19,1,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Informa...,False
4,186640,1Password for Linux,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",47,19,1,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: Confidence: A wil...,Targeted Clusters - Theme - Computing: Softwar...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2477,188060,QA Automation Developer,Zoocasa,Head Office,1,Open for Applications,Toronto,"Junior, Intermediate",12,19,1,"At Zoocasa, our mission is to ensure our clien...",Responsibilities: - Perform manual testing - C...,Required Skills: - Experience with manual test...,Targeted Clusters - Theme - Computing: Softwar...,False
2478,185591,Protein Engineering Lab,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate",7,19,2,About the Position We are currently seeking a ...,Your Job Supports recombinant protein producti...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Scientific Experim...,True
2479,185600,Computational Modelling,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",3,19,2,About the Position We are currently seeking a ...,Your Job Assists with the application and deve...,"Your Academics We want you to succeed, so you'...","Targeted Clusters - Theme - Data Analytics, St...",True
2480,185603,Software Development,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",4,19,3,About the Position Zymeworks Inc. is proud to ...,Your Job Work on a key software project in the...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Computing: Softwar...,True


In [74]:
df.iloc[1, "would_shortlist"] = True

ValueError: Can only index by location with a [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array]

In [74]:
df.to_csv("all_jobs.csv")

In [22]:
df = pd.read_csv('all_jobs.csv')

In [23]:
esther_shortllist = pd.read_csv("scrape_shortlist.csv")

In [27]:
victor_shortlist = pd.read_csv("victor-shortlist.csv")

In [28]:
victor_shortlist.drop(["Unnamed: 10"], axis = 1, inplace = True)

In [29]:
lists = [esther_shortllist, victor_shortlist]
sl = pd.concat(lists)

In [30]:
sl.drop_duplicates(subset=['ID'], inplace=True)

In [32]:
sl

,Unnamed: 0,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required
0,0.0,187777,Data Science,Achievers Inc,Divisional Office,2,Open for Applications,Toronto,"Intermediate, Senior",80,"Jan 19, 2021",8 month consecutive work term preferred,Data Science Co-Op Toronto Achievers delivers ...,"As a Data Science Co-op, you will: Design and ...",Qualifications: Enrolled in a Data Science or ...,Targeted Clusters - Theme - Computing: Softwar...,False
1,1.0,183894,Engineering Co-op,ActiveState Software Inc,Divisional Office,2,Open for Applications,Vancouver,"Intermediate, Senior",16,"Jan 19, 2021",4 month work term,Engineering Internships (Co-Ops) ActiveState P...,What You'll be Doing As an intern on one of ou...,"Requirements Experience with Linux, Window or ...",Targeted Clusters - Theme - Computing: Informa...,False
2,2.0,188750,Software Developer,AdHawk Microsystems,Divisional Office,1,Open for Applications,Kitchener,Senior,52,"Jan 19, 2021",4 month work term,AdHawk Microsystems is changing the way that h...,AdHawk is looking for software developers to h...,- Strong software development fundamentals - E...,Targeted Clusters - Theme - Computing: Softwar...,False
3,3.0,187277,Computer Vision Intern (Startup),Akasha Imaging,Divisional Office,2,Open for Applications,Remote in Canada,"Intermediate, Senior",153,"Jan 19, 2021",4 month work term,Akasha Imaging (http://akasha.im) is a VC-back...,- Independently implement algorithms for machi...,- Interest in understanding the fundamentals/p...,Targeted Clusters - Theme - Computing: Softwar...,False
4,4.0,182773,Software Development Engineering - Summer 2021...,Amazon.com Inc,Head Office,1,Open for Applications,NaN,"Intermediate, Senior",417,"Jan 19, 2021",4 month work term,"At Amazon, we hire the best minds in technolog...",KEY JOB RESPONSIBILITIES · Collaborate with ex...,BASIC QUALIFICATIONS · Programming experience ...,Targeted Clusters - Theme - Computing: Informa...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50,NaN,186745,NEW Data Science & Advanced Analytics,TD Bank Group,Analytics and Data Science,1,Open for Applications,Remote in Canada,"Intermediate, Senior",35,19-Jan-21,NaN,NaN,NaN,NaN,NaN,NaN
52,NaN,188232,NEW Junior Data Scientist,Triumf,Head Office,1,Open for Applications,Vancouver,"Intermediate, Senior",11,19-Jan-21,NaN,NaN,NaN,NaN,NaN,NaN
53,NaN,188244,NEW Quantum Machine Learning Researcher,Triumf,Head Office,1,Open for Applications,Vancouver,"Intermediate, Senior",9,19-Jan-21,NaN,NaN,NaN,NaN,NaN,NaN
54,NaN,187875,NEW Game Intelligence Analyst Intern,Ubisoft Toronto,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",10,19-Jan-21,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
sl.shape

(139, 17)

In [34]:
df["would_shortlist"] = df["ID"].isin(sl["ID"])

In [35]:
def in_shortlist(x):
    if (x in list):
        return True
    else:
        return False
    
df_in_shortlist = np.vectorize(in_shortlist)

In [233]:
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed,would_shortlist
0,185132,Accounting Analyst,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",32,19,1,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters - Theme - Accounting and Fin...,False,False
1,185136,Security,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",87,19,1,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment. ...,Targeted Clusters - Theme - Computing: Softwar...,False,False
2,186638,QA Intern,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",71,19,1,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Quality...,False,False
3,186639,Junior Developer,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",210,19,1,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Informa...,False,False
4,186640,1Password for Linux,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",47,19,1,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: Confidence: A wil...,Targeted Clusters - Theme - Computing: Softwar...,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2477,188060,QA Automation Developer,Zoocasa,Head Office,1,Open for Applications,Toronto,"Junior, Intermediate",12,19,1,"At Zoocasa, our mission is to ensure our clien...",Responsibilities: - Perform manual testing - C...,Required Skills: - Experience with manual test...,Targeted Clusters - Theme - Computing: Softwar...,False,False
2478,185591,Protein Engineering Lab,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate",7,19,2,About the Position We are currently seeking a ...,Your Job Supports recombinant protein producti...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Scientific Experim...,True,False
2479,185600,Computational Modelling,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",3,19,2,About the Position We are currently seeking a ...,Your Job Assists with the application and deve...,"Your Academics We want you to succeed, so you'...","Targeted Clusters - Theme - Data Analytics, St...",True,False
2480,185603,Software Development,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",4,19,3,About the Position Zymeworks Inc. is proud to ...,Your Job Work on a key software project in the...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Computing: Softwar...,True,False


In [36]:
df[df["would_shortlist"] == True]

,Unnamed: 0,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed,would_shortlist
25,25,187777,Data Science,Achievers Inc,Divisional Office,2,Open for Applications,Toronto,"Intermediate, Senior",55,19,3,Data Science Co-Op Toronto Achievers delivers ...,"As a Data Science Co-op, you will: Design and ...",Qualifications: Enrolled in a Data Science or ...,Targeted Clusters - Theme - Computing: Softwar...,False,True
29,29,183894,Engineering Co-op,ActiveState Software Inc,Divisional Office,2,Open for Applications,Vancouver,"Intermediate, Senior",8,19,1,Engineering Internships (Co-Ops) ActiveState P...,What You'll be Doing As an intern on one of ou...,"Requirements Experience with Linux, Window or ...",Targeted Clusters - Theme - Computing: Informa...,True,True
32,32,188750,Software Developer,AdHawk Microsystems,Divisional Office,1,Open for Applications,Kitchener,Senior,18,19,1,AdHawk Microsystems is changing the way that h...,AdHawk is looking for software developers to h...,- Strong software development fundamentals - E...,Targeted Clusters - Theme - Computing: Softwar...,False,True
51,51,187277,Computer Vision Intern (Startup),Akasha Imaging,Divisional Office,2,Open for Applications,Remote in Canada,"Intermediate, Senior",84,19,1,Akasha Imaging (http://akasha.im) is a VC-back...,- Independently implement algorithms for machi...,- Interest in understanding the fundamentals/p...,Targeted Clusters - Theme - Computing: Softwar...,False,True
58,58,182773,Software Development Engineering - Summer 2021...,Amazon.com Inc,Head Office,1,Open for Applications,NaN,"Intermediate, Senior",262,19,1,"At Amazon, we hire the best minds in technolog...",KEY JOB RESPONSIBILITIES · Collaborate with ex...,BASIC QUALIFICATIONS · Programming experience ...,Targeted Clusters - Theme - Computing: Informa...,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2369,2369,185126,Software Developer,Varicent,Development,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",36,19,1,Varicent is the leader in the Sales Performanc...,* Build new features for the REST API on top o...,* Knowledge and experience with RESTful APIs *...,Targeted Clusters - Theme - Computing: Softwar...,False,True
2400,2400,187337,"Associate Data Engineering - Platform, Infras...",Wattpad,Head Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",26,19,1,Wattpad is a global multiplatform entertainmen...,- Evolving Wattpad's data infrastructure - Imp...,- Solid understanding of data engineering best...,Targeted Clusters - Theme - Computing: Softwar...,False,True
2426,2426,187923,Data Engineering,Wonolo Inc,Head Office,1,Open for Applications,Remote in Canada,"Intermediate, Senior",26,19,1,Wonolo (Work Now Locally) is growing and disru...,- Iterate on existing CNN models and in-house ...,- Strong comprehension of statistical analysis...,Targeted Clusters - Theme - Computing: Softwar...,False,True
2448,2448,185105,Data Science Associate,XE.com,Divisional Office,1,Open for Applications,Newmarket,"Intermediate, Senior",17,19,1,The Opportunity This is a unique and exciting ...,Job Responsibilities: You will help build the ...,"Required Knowledge, Skills and Abilities Fluen...","Targeted Clusters - Theme - Data Analytics, St...",True,True


In [37]:
y_data = df["would_shortlist"]

In [43]:
x_data = df.drop("would_shortlist", inplace=False, axis=1)

In [45]:
x_data.to_csv("x_data.csv")
y_data.to_csv("y_data.csv")

In [8]:
x_data = pd.read_csv("x_data.csv").drop(["Unnamed: 0"], axis=1).reset_index(drop= True).fillna("")
y_data = pd.read_csv("y_data.csv").drop(["Unnamed: 0"], axis=1).reset_index(drop= True).fillna("")

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train.reset_index(drop= True, inplace=True)
y_train.reset_index(drop= True, inplace=True)

NameError: name 'X_train' is not defined

In [15]:
x_data["3_in_one"] = ""

In [16]:
for i in range(len(x_data["Work Term Duration"])):
    summary = x_data["Job Summary"].iloc[i]
    respo = x_data["Job Responsibilities"].iloc[i]
    skills =  x_data["Required Skills"].iloc[i]
    x_data["3_in_one"].iloc[i] = summary + respo + skills

c:\program files\python36\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [17]:
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score, f1_score, recall_score
np.random.seed(500)
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score


In [18]:
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
x_data["3_in_one"] = [entry.lower() for entry in x_data["3_in_one"]]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
x_data["3_in_one"]= [word_tokenize(entry) for entry in x_data["3_in_one"]]
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(x_data["3_in_one"]):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    x_data.loc[index,'text_final'] = str(Final_words)

KeyboardInterrupt: 

In [ ]:
x_data.to_csv("cleaned_all_data")

In [187]:
X_train.to_csv("3_in_onex_train_cleaned.csv")

In [104]:
import ast
word_count = {}
for i in range(len(x_data["Work Term Duration"])):
    for word in ast.literal_eval(x_data["text_final"].iloc[i]):
            word_count[word] = 1
    
        
    

In [13]:
x_data["text_final"].iloc[1]

KeyError: 'text_final'

In [106]:
temp = x_data.copy(deep=True)

In [107]:
temp['would_shortlist'] = y_data

In [19]:
temp = pd.read_csv('all_our_data_yay.csv')

In [21]:
x_data = temp.drop('would_shortlist', axis=1)
y_data = temp[['would_shortlist']]

In [108]:
temp.to_csv('all_our_data_yay.csv')

In [109]:
temp[temp['would_shortlist'] == True]

,Unnamed: 0.1,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed,3_in_one,text_final,would_shortlist
25,25,187777,Data Science,Achievers Inc,Divisional Office,2,Open for Applications,Toronto,"Intermediate, Senior",55,19,3,Data Science Co-Op Toronto Achievers delivers ...,"As a Data Science Co-op, you will: Design and ...",Qualifications: Enrolled in a Data Science or ...,Targeted Clusters - Theme - Computing: Softwar...,False,"[data, science, co-op, toronto, achievers, del...","['data', 'science', 'toronto', 'achiever', 'de...",True
29,29,183894,Engineering Co-op,ActiveState Software Inc,Divisional Office,2,Open for Applications,Vancouver,"Intermediate, Senior",8,19,1,Engineering Internships (Co-Ops) ActiveState P...,What You'll be Doing As an intern on one of ou...,"Requirements Experience with Linux, Window or ...",Targeted Clusters - Theme - Computing: Informa...,True,"[engineering, internships, (, co-ops, ), activ...","['engineering', 'internship', 'activestate', '...",True
32,32,188750,Software Developer,AdHawk Microsystems,Divisional Office,1,Open for Applications,Kitchener,Senior,18,19,1,AdHawk Microsystems is changing the way that h...,AdHawk is looking for software developers to h...,- Strong software development fundamentals - E...,Targeted Clusters - Theme - Computing: Softwar...,False,"[adhawk, microsystems, is, changing, the, way,...","['adhawk', 'microsystems', 'change', 'way', 'h...",True
51,51,187277,Computer Vision Intern (Startup),Akasha Imaging,Divisional Office,2,Open for Applications,Remote in Canada,"Intermediate, Senior",84,19,1,Akasha Imaging (http://akasha.im) is a VC-back...,- Independently implement algorithms for machi...,- Interest in understanding the fundamentals/p...,Targeted Clusters - Theme - Computing: Softwar...,False,"[akasha, imaging, (, http, :, //akasha.im, ), ...","['akasha', 'imaging', 'http', 'stealth', 'imag...",True
58,58,182773,Software Development Engineering - Summer 2021...,Amazon.com Inc,Head Office,1,Open for Applications,,"Intermediate, Senior",262,19,1,"At Amazon, we hire the best minds in technolog...",KEY JOB RESPONSIBILITIES · Collaborate with ex...,BASIC QUALIFICATIONS · Programming experience ...,Targeted Clusters - Theme - Computing: Informa...,False,"[at, amazon, ,, we, hire, the, best, minds, in...","['amazon', 'hire', 'best', 'mind', 'technology...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2369,2369,185126,Software Developer,Varicent,Development,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",36,19,1,Varicent is the leader in the Sales Performanc...,* Build new features for the REST API on top o...,* Knowledge and experience with RESTful APIs *...,Targeted Clusters - Theme - Computing: Softwar...,False,"[varicent, is, the, leader, in, the, sales, pe...","['varicent', 'leader', 'sale', 'performance', ...",True
2400,2400,187337,"Associate Data Engineering - Platform, Infras...",Wattpad,Head Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",26,19,1,Wattpad is a global multiplatform entertainmen...,- Evolving Wattpad's data infrastructure - Imp...,- Solid understanding of data engineering best...,Targeted Clusters - Theme - Computing: Softwar...,False,"[wattpad, is, a, global, multiplatform, entert...","['wattpad', 'global', 'multiplatform', 'entert...",True
2426,2426,187923,Data Engineering,Wonolo Inc,Head Office,1,Open for Applications,Remote in Canada,"Intermediate, Senior",26,19,1,Wonolo (Work Now Locally) is growing and disru...,- Iterate on existing CNN models and in-house ...,- Strong comprehension of statistical analysis...,Targeted Clusters - Theme - Computing: Softwar...,False,"[wonolo, (, work, now, locally, ), is, growing...","['wonolo', 'work', 'locally', 'grow', 'disrupt...",True
2448,2448,185105,Data Science A

In [22]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.10, random_state=42)

In [250]:
False == 0

True

In [23]:
df = pd.read_csv("3_in_onex_train_cleaned.csv")
shortlisted = pd.read_csv("scrape_shortlist.csv")

In [113]:
y_data.sum()

would_shortlist    128
dtype: int64

In [252]:
X_train

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed,3_in_one,text_final
0,187891,Finance Assistant,Ubisoft Toronto,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",17,19,1,Our Mission Ubisoft is committed to enriching ...,On being a Finance Assistant Intern at Ubisoft...,What it takes to make it Bachelor's degree in ...,Targeted Clusters - Theme - Accounting and Fin...,True,"[our, mission, ubisoft, is, committed, to, enr...","['mission', 'ubisoft', 'commit', 'enrich', 'pl..."
1,186006,Restorative Care/Recreation Aide,LaPointe-Fisher Nursing Home,Divisional Office,3,Open for Applications,Guelph,"Junior, Intermediate, Senior",13,19,1,LaPointe-Fisher is an accredited 92 bed nursin...,"As a Recreation Aide, you will support the rec...",Organized and self-starter Motivated and posit...,Targeted Clusters - Theme - Health Care: Thera...,True,"[lapointe-fisher, is, an, accredited, 92, bed,...","['accredited', 'bed', 'nursing', 'home', 'loca..."
2,186789,Engineering Intern,Wood Canada Limited,Wood Environment & Infrastructure,2,Open for Applications,Oakville,Intermediate,39,19,1,Wood Environment & Infrastructure Solutions ha...,The successful applicant will work closely wit...,The ideal candidate will have a good understan...,Targeted Clusters - Theme - Construction and I...,False,"[wood, environment, &, infrastructure, solutio...","['wood', 'environment', 'infrastructure', 'sol..."
3,187769,Actuarial Analyst,Brookfield,Brookfield Annuity,1,Open for Applications,Toronto,"Intermediate, Senior",27,19,1,Brookfield Annuity Company is a life insurance...,Pricing group annuities Analyze data and longe...,Prior work term experience in pensions and/or ...,"Targeted Clusters - Theme - Data Analytics, St...",True,"[brookfield, annuity, company, is, a, life, in...","['brookfield', 'annuity', 'company', 'life', '..."
4,187936,Back End Engineering - Core Products,Insticator,Divisional Office,1,Open for Applications,Kitchener,"Intermediate, Senior",6,19,1,About Us: Insticator increases engagement and ...,Work with the Core Products team on applicatio...,Basic Qualifications Solid experience developi...,Targeted Clusters - Theme - Computing: Informa...,False,"[about, us, :, insticator, increases, engageme...","['u', 'insticator', 'increase', 'engagement', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2228,185573,Technical Assistant,Peto MacCallum Ltd,Kitchener Office,2,Open for Applications,Kitchener,"Junior, Intermediate",39,19,1,"Peto MacCallum Ltd., an independent specialist...",Duties include field sampling and testing of c...,Qualified applicants will be in posession of a...,Targeted Clusters - Theme - Quality Assurance ...,False,"[peto, maccallum, ltd., ,, an, independent, sp...","['peto', 'maccallum', 'independent', 'speciali..."
2229,186228,Vehicle Connected Services,Nissan Canada Inc,Canadian Operations,1,Open for Applications,Mississauga,"Junior, Intermediate",4,19,3,Vehicle Connected Services Co-op Student Job P...,SUMMARY PROJECT DESCRIPTION: Perform testing a...,EDUCATION REQUIREMENTS: Eligible candidates mu...,Targeted Clusters - Theme - Computing: Hardwar...,True,"[vehicle, connected, services, co-op, student,...","['vehicle', 'connect', 'service', 'student', '..."
2230,188913,JK7 TechPubs Writer IP Routing,Nokia,Ottawa Office,1,Open for Applications,Ottawa,"Intermediate, Senior",0,19,1,We create the technology to connect the world....,"Job Responsibilities: Plan, research, write, a...",Qualifications Pursuing a Degree in Engineerin...,Targeted Clusters - Theme - Business Administr...,False,"[we, create, the, technology, to, connect, the...","['create', 'technology', 'connect', 'world', '..."
2231,188416,"Associate, Wood Gundy",Canadian Imperial Bank of Commerce,Wholesale Banking,1,Open for Applications,,"Junior, 

In [40]:
Tfidf_vect = TfidfVectorizer(max_features=15000)
Tfidf_vect.fit(X_train["text_final"])
Train_X_Tfidf = Tfidf_vect.transform(X_train['text_final'])
Test_X_Tfidf = Tfidf_vect.transform(X_test['text_final'])

In [26]:
print(Tfidf_vect.vocabulary_)

{'mission': 7885, 'ubisoft': 13112, 'commit': 2373, 'enrich': 4186, 'player': 9393, 'life': 7130, 'original': 8803, 'memorable': 7668, 'gaming': 5158, 'experience': 4511, 'create': 2917, 'world': 13951, 'people': 9135, 'express': 4556, 'explore': 4545, 'discover': 3576, 'new': 8339, 'possibility': 9535, 'toronto': 12839, 'found': 4990, 'contribute': 2724, 'many': 7476, 'brand': 1539, 'include': 6117, 'assassin': 817, 'far': 4678, 'honortm': 5791, 'tom': 12805, 'clancy': 2140, 'splinter': 11821, 'watch': 13685, 'team': 12477, 'develop': 3425, 'first': 4834, 'ip': 6545, 'starlink': 11940, 'battle': 1160, 'atlas': 907, 'lead': 7016, 'development': 3434, 'dog': 3717, 'legion': 7059, 'one': 8652, 'canada': 1746, 'top': 12828, 'employer': 4092, 'young': 14049, 'look': 7275, 'work': 13918, 'enthusiastic': 4213, 'expert': 4526, 'tackle': 12388, 'challenge': 1982, 'entertainment': 4208, 'beyond': 1272, 'offer': 8590, 'open': 8683, 'environment': 4240, 'bright': 1575, 'idea': 5969, 'chance': 199

In [27]:
print(Train_X_Tfidf)

  (2, 5966)	1.0
  (3, 12778)	0.8852906420791783
  (3, 6711)	0.4650381479481399
  (4, 8786)	1.0
  (5, 3664)	1.0
  (7, 11967)	0.8043690309332227
  (7, 6443)	0.594130004355569
  (8, 2133)	1.0
  (9, 7087)	1.0
  (10, 650)	1.0
  (11, 3180)	0.6377074979632906
  (11, 629)	0.770278616502756
  (12, 13918)	0.20722813506607332
  (12, 12602)	0.4247160381649678
  (12, 3826)	0.881290410116036
  (13, 12194)	0.8192099808218637
  (13, 6711)	0.5734936855117427
  (14, 10696)	0.9098174616300199
  (14, 6711)	0.41500865835920503
  (15, 11538)	0.7141847613420725
  (15, 10606)	0.6999572320269053
  (16, 12431)	1.0
  (17, 8249)	0.7206586302411211
  (17, 7084)	0.5057936457190716
  (17, 2872)	0.474156015051166


In [28]:
import itertools
import random
c_options = [0.2, 1, 4, 8]
kernal_options = ['linear', 'poly', 'rbf', 'sigmoid']
degree_options = [2,3,4] # go through only if poly
gamma_options = ['scale', 'auto'] # run only if not linear
coef0_options = [0,1,10,100] # run only poly sig
probability_options = [1,0]

my_combos = []


    


In [29]:
my_combos = []
for c in c_options:
    for p in probability_options:
        for k in kernal_options:
            if k == "linear":
                my_combos += [(c, k, 1, 'auto', 0, p)]
            else:
                for g in gamma_options:
                    if k == "rbf":
                        my_combos += [(c, k, 1, g, 0, p)]
                    else:
                        for co in coef0_options:
                            if k == "sigmoid":
                                my_combos += [(c, k, 1, g, co, p)]
                            else:
                                for d in degree_options:
                                    my_combos += [(c, k, d, g, co, p)]
                                    
best_5_combos = []
random_combos = random.shuffle(my_combos)


                

In [30]:
def update_best_score(c, best_combo, best_score):
    SVM = svm.SVC(C=c[0], kernel=c[1], degree=c[2], gamma=c[3], coef0 = c[4], probability = c[5], class_weight = 'balanced')
    SVM.fit(Train_X_Tfidf,Train_Y)
    # predict the labels on validation dataset
    predictions_SVM = SVM.predict(Test_X_Tfidf)
    # Use accuracy_score function to get the accuracy
    my_f1_score = f1_score(predictions_SVM, Test_Y)*100
    print("SVM Accuracy Score -> ",my_f1_score)
    if (my_f1_score > best_score):
        best_score = my_f1_score
        best_combo = c
    return best_combo, best_score

In [31]:
Tfidf_vect = TfidfVectorizer(max_features=None)
Tfidf_vect.fit(X_train['text_final'])
cross_val_Tfidf = Tfidf_vect.transform(X_train['text_final'])

In [51]:
best_combo = {}

for c in [(8, 'poly', 2, 'scale', 1, 1)]:
    # Classifier - Algorithm - SVM
    # fit the training dataset on the classifier
    SVM = svm.SVC(C=c[0], kernel=c[1], degree=c[2], gamma=c[3], coef0 = c[4], probability = c[5], class_weight = 'balanced')
    SVM.fit(Train_X_Tfidf,y_train['would_shortlist'])
    scores = cross_val_score(SVM, cross_val_Tfidf, y_train['would_shortlist'], cv=10, scoring='recall')
    # predict the labels on validation dataset
    #predictions_SVM = SVM.predict(Test_X_Tfidf)
    # Use accuracy_score function to get the accuracy
    #my_recall_score = recall_score(predictions_SVM, Test_Y)*100
    print("SVM Accuracy Score -> ",scores.mean())
    best_combo[scores.mean()] = (c, scores.std())

SVM Accuracy Score ->  0.39545454545454545


In [52]:
SVM

SVC(C=8, class_weight='balanced', coef0=1, degree=2, kernel='poly',
    probability=1)

In [55]:
with open('svm.pk', 'wb') as fin:
    pickle.dump(SVM, fin)

In [54]:
best_combo

{0.39545454545454545: ((8, 'poly', 2, 'scale', 1, 1), 0.15897542706201498)}

In [38]:
Train_X_Tfidf

<20x14096 sparse matrix of type '<class 'numpy.float64'>'
	with 25 stored elements in Compressed Sparse Row format>

In [43]:
import pickle

In [44]:
with open('vectorizer.pk', 'wb') as fin:
    pickle.dump(Tfidf_vect, fin)

In [45]:
best_combo_recall = {}

for c in my_combos[:100]:
    # Classifier - Algorithm - SVM
    # fit the training dataset on the classifier
    SVM = svm.SVC(C=c[0], kernel=c[1], degree=c[2], gamma=c[3], coef0 = c[4], probability = c[5], class_weight = 'balanced')
    SVM.fit(Train_X_Tfidf,y_train['would_shortlist'])
    # predict the labels on validation dataset
    predictions_SVM = SVM.predict(Test_X_Tfidf)
    # Use accuracy_score function to get the accuracy
    my_recall_score = recall_score(predictions_SVM, y_test['would_shortlist'])*100
    print("SVM Accuracy Score -> ",my_recall_score)
    best_combo_recall[c] = my_recall_score

SVM Accuracy Score ->  40.0
SVM Accuracy Score ->  4.819277108433735


C:\Users\esthe\AppData\Roaming\Python\Python36\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


SVM Accuracy Score ->  0.0
SVM Accuracy Score ->  57.14285714285714
SVM Accuracy Score ->  55.55555555555556


C:\Users\esthe\AppData\Roaming\Python\Python36\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


SVM Accuracy Score ->  0.0
SVM Accuracy Score ->  40.0
SVM Accuracy Score ->  42.10526315789473
SVM Accuracy Score ->  4.819277108433735
SVM Accuracy Score ->  4.819277108433735
SVM Accuracy Score ->  40.0
SVM Accuracy Score ->  28.57142857142857
SVM Accuracy Score ->  50.0
SVM Accuracy Score ->  4.819277108433735
SVM Accuracy Score ->  4.819277108433735
SVM Accuracy Score ->  4.819277108433735
SVM Accuracy Score ->  4.819277108433735
SVM Accuracy Score ->  60.0


...

C:\Users\esthe\AppData\Roaming\Python\Python36\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


SVM Accuracy Score ->  0.0
SVM Accuracy Score ->  40.0
SVM Accuracy Score ->  4.819277108433735


C:\Users\esthe\AppData\Roaming\Python\Python36\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


SVM Accuracy Score ->  0.0
SVM Accuracy Score ->  40.0
SVM Accuracy Score ->  57.14285714285714
SVM Accuracy Score ->  4.819277108433735
SVM Accuracy Score ->  40.0
SVM Accuracy Score ->  50.0
SVM Accuracy Score ->  4.819277108433735
SVM Accuracy Score ->  50.0
SVM Accuracy Score ->  43.75
SVM Accuracy Score ->  47.05882352941176
SVM Accuracy Score ->  0.0


C:\Users\esthe\AppData\Roaming\Python\Python36\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [47]:
best_combo_recall

{(0.2, 'poly', 4, 'auto', 10, 0): 40.0,
 (8, 'sigmoid', 1, 'scale', 100, 1): 4.819277108433735,
 (0.2, 'sigmoid', 1, 'auto', 100, 0): 0.0,
 (4, 'poly', 4, 'auto', 100, 1): 57.14285714285714,
 (8, 'sigmoid', 1, 'scale', 0, 0): 55.55555555555556,
 (0.2, 'poly', 3, 'auto', 10, 1): 0.0,
 (1, 'poly', 3, 'auto', 100, 1): 40.0,
 (8, 'poly', 2, 'auto', 100, 0): 42.10526315789473,
 (8, 'rbf', 1, 'auto', 0, 1): 4.819277108433735,
 (4, 'sigmoid', 1, 'auto', 0, 1): 4.819277108433735,
 (1, 'poly', 3, 'auto', 100, 0): 40.0,
 (0.2, 'poly', 3, 'scale', 0, 0): 28.57142857142857,
 (0.2, 'poly', 4, 'auto', 100, 0): 50.0,
 (8, 'sigmoid', 1, 'auto', 1, 1): 4.819277108433735,
 (4, 'sigmoid', 1, 'auto', 100, 0): 4.819277108433735,
 (4, 'sigmoid', 1, 'scale', 100, 1): 4.819277108433735,
 (1, 'poly', 2, 'auto', 1, 1): 4.819277108433735,
 (4, 'rbf', 1, 'scale', 0, 0): 60.0,
 (0.2, 'poly', 3, 'auto', 1, 0): 0.0,
 (0.2, 'poly', 2, 'auto', 0, 0): 0.0,
 (8, 'poly', 2, 'scale', 1, 1): 66.66666666666666,
 (8, 'poly',